Importing relevant libraries

In [1]:
import pandas as pd
from datetime import datetime
import pyeto
import numpy as np
import ast       
import math
import xlrd
from ast import literal_eval  
from pandas import DataFrame
from scipy.interpolate import interp1d
import dateutil     #dateutil module provides powerful extensions to the standard datetime module
from dateutil import parser  #This module offers reads the given date in string and convert it to date format or timestamps,it represent a date and/or time from most known formats 
import os
from tkinter import filedialog, messagebox
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm
import json
from IPython.display import display, Markdown, HTML

# note that pyeto is available here https://github.com/woodcrafty/PyETo.git
from pyeto import fao

%matplotlib inline

math.exp = np.exp
math.pow = np.power
math.sqrt = np.sqrt

# Choosing the scenario to be analysed

Two scenarios have been used for the analysis:

- Average-case scenario wwhich uses the average monthly precipitation and temperature data for the last 10 years

- Worst-case scenario which picks the months with the lowest rainfall and highest temperatures for the last 10 years

The input files for the average scenario and worst-case (drought) scenario are provided

A future scenario input file can be generated accordingly by using projections for precipitation and temperature

Select the scenario to be analysed below as follows:
- 0 for average scenario
- 1 for worse-case scenario
- 2 for future-case scenario

In [2]:
#select scenario for analysis
#0 for average scenario
#1 for worst case scenario
s = 1

Reading in the input files

In [3]:
if s == 1:
    df = pd.read_csv('worst_case_input.csv')
elif s == 2:
    df = pd.read_csv('future_case_input.csv')
else:
    df = pd.read_csv('average_case_input.csv')

df.head(5)

# Determining the reference evapotranspiration (ETo)

The function for estimating the parameters for calculating reference evapotranspiration requires the following inputs:
- Latitude
- Longitude
- Elevation
- Monthly Wind speed at 2m
- Monthly Solar irradiation
- Monthly minimum temperature
- Monthly maximum temperature
- Maximum monthly temperature

These inputs are used for estimating the parameters for the Penman-Monteith equation for evapotranspiration which are:
- Net radiation at the crop surface
- Air temperature at 2m
- Wind speed at 2m
- Saturation vapour pressure
- Actual vapour pressure
- Saturation vapour pressure deficit
- Slope vapour pressure curve
- Psychrometric constant

In [4]:
#Defining function for estimating parameters required to calculate monthly reference evapotranspiration
def evap_i(lat,elev,wind,srad,tmin,tmax,tavg,month):
    if month ==1:
        J = 15
    else:
        J = 15 + (month-1)*30
        
    latitude = pyeto.deg2rad(lat) #converting latitude values from degrees to radians
    atmosphericVapourPressure = pyeto.avp_from_tmin(tmin) #obtaining atmospheric vapour pressure from t_min
    saturationVapourPressure = pyeto.svp_from_t(tavg) #obtaining saturation vapour pressure from t_avg
    ird = pyeto.inv_rel_dist_earth_sun(J) #obtaining irradiance based on distance from sun?
    solarDeclination = pyeto.sol_dec(J) #solar declination for various months?
    sha = [pyeto.sunset_hour_angle(l, solarDeclination) for l in latitude]
    extraterrestrialRad = [pyeto.et_rad(x, solarDeclination,y,ird) for x, y in zip(latitude,sha)] #extraterrestial radiation
    clearSkyRad = pyeto.cs_rad(elev,extraterrestrialRad) #clear sky radiation
    netInSolRadnet = pyeto.net_in_sol_rad(srad*0.001, albedo=0.23) #net shortwave irradiation
    netOutSolRadnet = pyeto.net_out_lw_rad(tmin, tmax, srad*0.001, clearSkyRad, atmosphericVapourPressure) #net longwave radiation
    netRadiation = pyeto.net_rad(netInSolRadnet,netOutSolRadnet) #net radiation
    tempKelvin = pyeto.celsius2kelvin(tavg) #converting average temperature to Kelvin
    windSpeed2m = wind #wind speed
    slopeSvp = pyeto.delta_svp(tavg) #calculating slope vapour pressure
    atmPressure = pyeto.atm_pressure(elev) #calculating atmospheric pressure from elevation value
    psyConstant = pyeto.psy_const(atmPressure) #calculating psychrometric constant from atmospheric pressure
    
    #the function to use FAO Penman-Monteith equation and return the value of reference evapotranspiration
    return pyeto.fao56_penman_monteith(netRadiation, tempKelvin, windSpeed2m, saturationVapourPressure, 
                                       atmosphericVapourPressure, slopeSvp, psyConstant, shf=0.0)


#initiating the value of reference evapotranspiration
for i in range(1,13):
    df['ETo_{}'.format(i)]=0  #To make sure that it is reset to zero

#calling the function to calculate ETo for each row for each month 
for i in range(1,13):
    df['ETo_{}'.format(i)] = evap_i(df['lat'],df['elevation'],df['wind_{}'.format(i)],df['srad_{}'.format(i)], 
                                    df['tmin_{}'.format(i)],df['tmax_{}'.format(i)],df['tavg_{}'.format(i)],i)


In [5]:
df.head(5)

# Estimating monthly crop evapotranspiration (ETc)

Depending on the crop coefficient at different stages of its growth and the growth calendar, a crop coefficient is assigned to each month from January to december

Crop evapotranspiration is calculated as a product of reference evapotranspiration and crop coefficient.

In [6]:
# Estimate monthly crop evaropotransoration ETc
for i in range(1,13):
    df['ETc_{}'.format(i)] = df['ETo_{}'.format(i)] * df['kc_{}'.format(i)]

df.head(5)

# Determining the effective rainfall

Effective rainfall is obtained by multiplying the monthly rainfall with a multiplication factor which is determined based on climatic conditions, soil water holding capacity, crop rooting depth and whether the crop is grown on dryland or wetland.

In [7]:
#calculating the effective rainfall
#the rainfall received is multiplied with relevant factor
#for this case maize is considered a shallow rooted crop

#Initiate
for i in range(1,13):
    df['eff_{}'.format(i)]=0


#the effective rainfall kept with bounds of 0.0001 to ETc value to avoid negative numbers when the two are
#subtracted
for i in range(1,13):
    df['eff_{}'.format(i)] = df['c_shallow'] * df['prec_{}'.format(i)]
    df.loc[df['eff_{}'.format(i)] < 0, 'eff_{}'.format(i)] = 0.0001
    df.loc[(df['eff_{}'.format(i)] >= df['prec_{}'.format(i)]), 'eff_{}'.format(i)] = df['prec_{}'.format(i)]
    df.loc[(df['eff_{}'.format(i)] >= df['ETc_{}'.format(i)]), 'eff_{}'.format(i)] = df['ETc_{}'.format(i)]
    
df.head(5)

# Calculating Net Irrigation water requirements

The net irrigation water requirement is calculated by subtraction the effective rainfall from the crop evapotranspiration which is taken to be equivalent to crop water needs.

In [8]:
#Net Irrigation requirements (IRn) (mm/month)
#monthly ETc - monthly effective rainfall
for i in range (1,13):
    df['IRn_{}'.format(i)]= df['ETc_{}'.format(i)]*30 - df['eff_{}'.format(i)]*30

df.head(5)

In the code below the irrigation water demand is converted mm/month to litres/ha/day

In [9]:
#Peak Crop Water Requirements (PCWR)

# Converting IRn into (m3/ha per month) 
for i in range (1,13):
    df['IRn_{}'.format(i)] *= 10    #0.001*10000 --- 1 hectare*1mm
    df['Monthly_IRn_{}'.format(i)] = df['IRn_{}'.format(i)]
    
# Converting IRn into (m3/ha per day)
for i in range (1,13):
    df['IRnd_{}'.format(i)] = df['IRn_{}'.format(i)] / 30
    
# Peak crop water requirement (PCWR) taken to be equal to net irrigation requirement
for i in range (1,13):
    df['PCWR_{}'.format(i)] = df['IRnd_{}'.format(i)] 
    
# Converting PCWR into  l/s/ha "Duty" - litre/second/hectare
for i in range (1,13):
    df['PCWR_{}'.format(i)] *= 0.012 #1000/86400 (seconds in a day)
    

df.head(5)


The code below determine the maximum daily irrigation water demand in litres/ha/day
This is given by the month with the maximum demand

In [10]:
#max pcwr
df['pcwr_max']=df.filter(like='PCWR_').max(axis=1) 
df['pwcr_per_day'] = df['pcwr_max']*60*60*24 #pwcr in litres/day/hectare

df.head(5)

This code determines the month with the maximum irrigation water demand

In [11]:
#which month has the max pcwr?
x = {}
x['Max_PCWR']=df.filter(like='PCWR_').idxmax(axis=1)

df['pcwr_month'] = x['Max_PCWR'].str[5:]

df.head(5)

# Calculating the gross water demand

The gross water demand accounts for water losses and inefficiencies of distribution method used.

The efficiencies are categorised to:
- Conveyance efficiency which is depended on soil type and size of the farm. In this case it is taken to be 0.95 by assuming used of pipes for water distribution.
- Application efficiency which depends on the method of irrigation used. In this case this was taken to be 0.9 by assuming the use of drip irrigation.

In addition, it was assumed that water pumping will take place for 8 hours a day therefore the 24 hour demand should be catered for within these hours of pumping.

In [12]:
#Peak Water Demand (PWD) in l/s 

# PWD = PCWR / Irrigation efficiency(IrrEff) 
# IrrEff = Field Application Efficiency (aeff) * Distribution Efficiency (deff)*100 
# deff = (Conveyance efficiency + field canal efficiency)
# deff: 0.95 (all scenarios)
# aeff: 0.6 (Surface Irr), 0.75 (Sprinkler Irr), 0.9 (Drip Irr)

#efficiency of drip irrigation will be used 

pumping_hours_per_day=8    # Assumption  
deff= 0.95                 # Assumption
aeff= 0.9                 # Assumption

#peak water demand per day in l/s/ha
for i in range (1,13):
    df["PWD_{}".format(i)]= df["PCWR_{}".format(i)]*24/(pumping_hours_per_day*aeff*deff)

df.head(5)

# Calculating the required pump diameter

The required pump diameter depends on the maximum volumetric flow rate and the maximum allowable water velocity. In this case the maximum allowable water velocity was taken to be 1.5m/s

In [13]:
#pump diameter depending on peak water demand
#diameter in metres

max_vel = 1.5 #maximum allowable water velocity
df['PWD_max']=df.filter(like='PWD_').max(axis=1)

df['diam'] = np.sqrt((4*df['PWD_max']/1000)/(3.142*max_vel))



df.head(5)

# Calculating the total dynamic head

The total dynamic head incorporates the depth of water, fricition head and required pressure head.

Friction head is calculated according to Darcy formula which considers friction factor of the pipe material, length of the pipe, diameter of the pipe and acceleration due to gravity. The friction factor is taken to be 0.013 by assuming use of PVC pipes.

Pressure head is taken to be 2m.

In [14]:
#Total dynamic head (TDH) for ground and surface water sources

kf = 0.013 #friction factor

# #friction head of groundwater
frict_head_gw = (4*kf*8*df["gw_depth"]*(df["PWD_max"]/1000)**2)/(((df["diam"])**5)*(3.142**2)*9.81)
pres_head_gw = 2 #pressure head



pres_head_sw = 2
frict_head_sw = (4*kf*8*df['sw_dist']*(df['PWD_max']/1000)**2)/(((df['diam'])**5)*(3.142**2)*9.81)

#groundwater dynamic head
df['tdh_gw']=df['gw_depth'] + pres_head_gw + frict_head_gw



#checking if water us flowing downhill or uphill
df['tsh'] = df['sw_depth'] - df['elevation']

#surface water dynamic head
df.loc[df['tsh'] <= 0, 'tdh_sw'] = (abs(df['sw_depth'] - df['elevation'])) + pres_head_sw + frict_head_sw
df.loc[df['tsh'] > 0, 'tdh_sw'] = pres_head_sw + frict_head_sw


df.head(5)

# Estimating power demand (kW)

Power demand depends on the demanded volumetric flow rate, total dynamic head and the pumping system efficiency.

The pumping system efficiency was assumed to be 57%

In [15]:
#Setting the default value for these parameters
for i in range (1,13):
    df['PD_E_gw_{}'.format(i)]=0      #PD_E_gw: Peak Demand (kw) using electric powered pump for ground water
    df['PD_E_sw_{}'.format(i)]=0      #PD_E_sw: Peak Demand (kw) using electric powered pump for surface water 

    
pump_plant_eff=0.57

#calculating the monthly peak (kW) demand
for i in range (1,13):
    PWD = 'PWD_{}'.format(i)
    PD_E_gw = 'PD_E_gw_{}'.format(i)
    PD_E_sw = 'PD_E_sw_{}'.format(i)

    df[PD_E_gw]=(9.81*(df[PWD]/1000)*df['tdh_gw'])/pump_plant_eff #Dividing by 1000 to obtain power in kW
    df[PD_E_sw]=(9.81*(df[PWD]/1000)*df['tdh_sw'])/pump_plant_eff

df.head(5)

Code below decides whether to use surface water or ground water depending on which one has the least power requirements

In [16]:
df['Max_PD_E_gw']=df.filter(like='PD_E_gw_').max(axis=1) #highest power for gw
df['Max_PD_E_sw']=df.filter(like='PD_E_sw_').max(axis=1)  #highest power for sw
df['PD_E'] = np.minimum.reduce(df[['Max_PD_E_gw', 'Max_PD_E_sw']].values, axis=1) #choosing lowest of the two

#noting whether surface water or groundwater was used
#1 if surface water is chosen and 0 if groundwater is chosen
df.loc[df['Max_PD_E_sw'] < df['Max_PD_E_gw'], 'sw'] = 1
df.loc[df['Max_PD_E_sw'] > df['Max_PD_E_gw'], 'sw'] = 0

df.head(5)

# Estimating the electricity demand for irrigation

In [17]:
#Estimating the yearly electricity demand

#total sum of monthly
#months assumed to have same length so this can be added directly
df['PD_E_gw_sum']=df.filter(like='PD_E_gw_').sum(axis=1) #sum power for gw
df['PD_E_sw_sum']=df.filter(like='PD_E_sw_').sum(axis=1)  #sum power for sw

df['PD_sum'] = np.minimum.reduce(df[['PD_E_gw_sum', 'PD_E_sw_sum']].values, axis=1) #choosing the lowest power sum

#monthy energy demand
for i in range (1,13):
    month_demand = 'month_demand_{}'.format(i)
    power_sw = 'PD_E_sw_{}'.format(i)
    power_gw = 'PD_E_gw_{}'.format(i)
    df.loc[df['Max_PD_E_sw'] < df['Max_PD_E_gw'], month_demand] = df[power_sw]*8*30
    df.loc[df['Max_PD_E_sw'] > df['Max_PD_E_gw'], month_demand] = df[power_gw]*8*30
    

#total annual energy consumed in irrigation
df['Annual_elec_demand'] = df['PD_sum'] * 8 * 30

df.head(5)

# Determining the solar potential for the month with max power demand

In [18]:
#determining solar potential of month with max demand

df['etc_max']=df.filter(like='IRn_').idxmax(axis=1)

d = df['etc_max']

x = d.str[4:]

x
n = x.astype(int)
 
start_pv = df.columns.get_loc('pvout_1')
pos = start_pv + (n)
pos

colname = df.columns[pos]


for i in range(len(df)):
    df['PV'] = df.loc[:][colname[i]]

df.head(5)

# Sizing the required PV

In [19]:
#PV size = 1.2 x Max Energy Demand (kWh)/pv potential (kWh/kWp)
#Result scaled by 20\% for safety as the monthly PVOUT value is an average

df['pv_kw'] = (1.2*df['PD_E']*8*30)/df['PV']


df.head(5)

# Annual energy supplied and system utilisation factor

In [20]:
#Total energy produced by system per month
#Energy (kWh) = PV Peak Power (kWp) * solar potential (kWh/kWp) 
for i in range (1,13):
    df['E_supply_{}'.format(i)] = df['pv_kw']*df['pvout_{}'.format(i)] #monthly energy supplied
    
#Total energy produced by the system annually
#Total energy supplied is given by the sum of the monthly supplies
df['Annual_Energy']=df.filter(like='E_supply_').sum(axis=1)


#utilisation factor
df['utilisation_1'] = 100 * df['Annual_elec_demand']/df['Annual_Energy']


df.head(5)

# Potential for using excess energy to connect neighbouring households to Tier 3 electricity

In [21]:
#Excess energy per month i.e. the energy not used for irrigation

for i in range (1,13):
    E_Excess = 'E_Excess_{}'.format(i)
    E_Supply = 'E_supply_{}'.format(i)
    demand = 'month_demand_{}'.format(i)
    df[E_Excess] = df[E_Supply] - df[demand]

    
df.head(5)

In [22]:
#electrification demand

#https://openknowledge.worldbank.org/entities/publication/a896ab51-e042-5b7d-8ffd-59d36461059e
#https://openknowledge.worldbank.org/server/api/core/bitstreams/248a7205-e926-5946-9025-605b8035ad95/content
tier1 = 4.5 #annual household consumption for tier 1 in kWh
tier2 = 73
tier3 = 365
tier4 = 1250
tier5 = 3000
#tier 3 demand per month = 365kWh/12
#assuming 4 people per household
#demand = area being connected * density of people without electricity * demand per/household/month

#connecting 1km^2 around the SPIS
df['tier3'] = df['no_access']*1*tier3/(4*12)

    
df.head(5)

In [23]:
#determining how much of the SPIS excess energy can be consumed by the households/per month
for i in range (1,13):
    E_Excess = 'E_Excess_{}'.format(i)
    amount_supplied = 'amount_supplied_{}'.format(i)
    Tier3 = 'tier3'
    df.loc[df[E_Excess] > df[Tier3], amount_supplied] = df[Tier3] #if supply greater than demand
    df.loc[df[E_Excess] < df[Tier3], amount_supplied] = df[E_Excess] #if demand greater than supply
    
df.head(5)

# Impact of using SPIS for electrification

In [24]:
#total energy supplied to electrified homes annually
df['total_supplied']=df.filter(like='amount_supplied_').sum(axis=1)

#number of households getting tier 3
df['tier3_hh'] = round(df['total_supplied']/tier3)

#new utilisation factor
#adding up energy supplied for electrification and energy used for irrigation
df['utilisation_2'] = 100*(df['Annual_elec_demand'] + df['total_supplied'])/df['Annual_Energy']

df.head(5)

# Estimating SPIS cost

In [25]:
#system cost
#cost of submersible pump + solar panels = $1109/kW #https://www.waterpumps.co.ke/solar-water-pumps-prices-in-kenya.html
#cost of surface pump + solar panels = $1295/kW #https://www.waterpumps.co.ke/solar-water-pumps-prices-in-kenya.html
#cost drilling a borehole = $833 + $54*depth #https://www.waterlink.co.ke/2021/05/07/borehole-drilling-cost-in-kenya-2022/#:~:text
#cost of water transmission pipeline = $1.14*distance #https://grekkon.com/hdpe-pipes-in-kenya/

#cost excluding the boreholes or the water transmission pipe
df.loc[df['sw'] == 1, 'cost'] = 1295*df['PD_E'] #if using surface water
df.loc[df['sw'] == 0, 'cost'] = 1109*df['PD_E'] #if using ground water

df.head(5)

# Generating output file

In [26]:
#removing protected areas
x = df.loc[(df['protected'] == 1)] 

In [27]:
#filtering columns of interest
#county, main id, area,longitude, latitude
#pcwr, peak power demand, pv size, energy demand, utilisation 1, electricity supplied, 
#utilisation 2 (after electrification), cost, tier3_hh (number of households that can be electrified to tier 3)
out = x.filter(items=['COUNTY','Main ID','area', 'lon','lat','pwcr_per_day', 'PD_E','pv_kw', 'Annual_elec_demand',
                'total_supplied','utilisation_1','utilisation_2','cost','tier3_hh'])
out.head(5)

# Saving the output file

In [28]:
#saving
if s == 1:
    out.to_csv('worst_case_output.csv', index = False)
elif s == 2:
    out.to_csv('future_case_output.csv', index = False)
else:
    out.to_csv('average_case_output.csv', index = False)
    
out.head(5)